# Test models

## Test scoring model

In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

data_path='../../DATASETS/IT_TEXTS'

Xy = pd.read_feather(f'{data_path}/PREPROCESSING/Xy.feather')
Xy.fillna('', inplace=True)
#X, y = Xy[['title', 'summary']], Xy['class']
#del Xy
#
train, test = train_test_split(Xy, test_size=.2, random_state=0)

In [10]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained(f'{data_path}/MODELS/checkpoint-44000')

In [11]:
def classificate(x):
    t, s = x.title, x.summary
    tokens = tokenizer.encode_plus(t, s, return_tensors="pt")
    classification_logits = model(**tokens)[0]
    results = torch.softmax(classification_logits, dim=1).tolist()[0]
    return results[1]

In [12]:
from tqdm.notebook import tqdm

In [39]:
_ = test[:10000]
_['score'] = _.progress_apply(classificate, axis=1)

RuntimeError: The size of tensor a (836) must match the size of tensor b (512) at non-singleton dimension 1

In [30]:
_['score'] = _['score'].apply(round)

<ipython-input-30-2d2f8f22b326>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _['score'] = _['score'].apply(round)


In [35]:
precision_recall_fscore_support(_['class'], _['score'], average='macro')

(0.623342175066313, 0.5988660524450744, 0.607843137254902, None)